In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

#https://www.tripadvisor.com/Restaurants-g293986-Amman_Amman_Governorate.html
"""
soup.find_all('a', {'class':'property_title'})
soup.find_all('span', {'class':'reviewCount'})
soup.find_all('div', {'class':'popIndex rebrand popIndexDefault'}) ranking
soup.find_all('span', {'class':'item price'})
soup.find_all('a', {'class':'item cuisine'})

"""
                          
property_titles = []
review_count = []
rankings = []
prices = []
cuisines = []
url2 = []

for offset in range(0, 19):
    url = 'https://www.tripadvisor.com/Restaurants-g293986-oa' + str(offset) + '-Amman_Amman_Governorate.html#EATERY_OVERVIEW_BOX'

    r = requests.get(url)
#    soup = BeautifulSoup(r.text, "html.parser")
    time.sleep(0.05)
    soup = BeautifulSoup(r.text,"lxml")
#    li = soup.find_all('div', {'id':'EATERY_SEARCH_RESULTS'}) # all restaurants in listing
    
#    for restaurant in soup.find_all(')
    for link in soup.find_all('a', {'class':'property_title'}):
        title = link.text
        property_titles.append(title.strip('\n'))
        
    for link in soup.find_all('span', {'class':'reviewCount'}):
        count = link.text
        review_count.append(count.strip('\n'))
    
    for link in soup.find_all('div', {'class':'popIndex rebrand popIndexDefault'}):
        rank = link.text
        rankings.append(rank.strip('\n'))
        
    # some restaurants don't have a price, have to figure this out
    for link in soup.find_all('span', {'class':'item price'}):
        price = link.text
        prices.append(price.strip('\n'))
       
    # this isn't really working as of now -- it collects all cuisine tags, not just one for each restaurant
    #for link in soup.find_all('a', {'class':'item cuisine'}):
        #cuisine = link.text
        #print(cuisine)
        #cuisines.append(cuisine.strip('\n'))
        
    
    #for this I am not sure what you were thinking about it, but what if we take all of the tag cuisines 
    #for each restaurant as a restaurant may be in more than one category and then perhaps use these also for 
    #clustering or anything ?? 
    
    
    t = []
    for link in soup.find_all('div', {'class':'cuisines'}):
        for k in link.find_all('a',{'class':'item cuisine'}):
            t.append(k.text)
        cuisines.append(t)
        t = []

    #added url so that we can use these links to move through all the pages in tripadvisor_restaurant.py 
    # "https://tripadvisor.com/ + url_of_rest
    for link in soup.find_all('a',{'class':'property_title'}):
        url_of_rest = link['href']
        url2.append(url_of_rest.strip('\n'))

#columns = {'property_title':property_titles, 'review_count':review_count, 'ranking':rankings}
columns = {'property_title':property_titles, 'review_count':review_count, 'url2':url2, 'ranking':rankings, 'cuisines':cuisines}
df = pd.DataFrame(columns)
print(df[:10])

                                            cuisines  \
0                               [American, Barbecue]   
1        [Pizza, Arabic, Halal, Gluten Free Options]   
2                            [Middle Eastern, Halal]   
3  [Lebanese, Mediterranean, Middle Eastern, Hala...   
4  [Mediterranean, Middle Eastern, Halal, Gluten ...   
5              [Steakhouse, American, International]   
6  [Lebanese, Mediterranean, Middle Eastern, Hala...   
7  [American, Vegetarian Friendly, Vegan Options,...   
8  [Fast Food, Mediterranean, Middle Eastern, Veg...   
9  [Fast Food, Mediterranean, Middle Eastern, Veg...   

              property_title                          ranking    review_count  \
0                    Brisket   #1 of 545 Restaurants in Amman    268 reviews    
1            Pizza Roma Cafe   #2 of 545 Restaurants in Amman    300 reviews    
2             Habibah Sweets   #3 of 545 Restaurants in Amman    665 reviews    
3      Fakhreldin Restaurant   #4 of 545 Restaurants in Amm

In [5]:
print(url2)

['/Restaurant_Review-g293986-d7267482-Reviews-Brisket-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d3389330-Reviews-Pizza_Roma_Cafe-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d2084697-Reviews-Habibah_Sweets-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d1371269-Reviews-Fakhreldin_Restaurant-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d2406112-Reviews-Sufra_Restaurant-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d3989474-Reviews-V_Lounge_Restaurant-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d1641029-Reviews-Tawaheen_al_Hawa-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d7761403-Reviews-Chestnut_Restaurant_Pub-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d1918356-Reviews-Hashem-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d3261923-Reviews-Abu_Jbara-Amman_Amman_Governorate.html', '/Restaurant_Review-g293986-d10005312-Reviews-Melograno-Amman_Amman_Governorate.htm

In [22]:
len(url2)

570

In [8]:
import csv
with open('restaurants.csv', 'w') as csvfile:
    df.to_csv('restaurants.csv', sep='\t', encoding='utf-8')
    #df.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.')
    #fieldnames = ['first_name', 'last_name']
    #writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#df.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.')

In [30]:
with open("links.csv", "w",newline="") as f:
    writer = csv.writer(f)
    for i in url2:
        writer.writerow([i])
    #writer = csv.writer(f, dialect='excel')
    #writer.writerows(url2)